In [5]:
#https://qiskit.org/documentation/nature/tutorials/01_electronic_structure.html

from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver

if True:
    driver = PySCFDriver(
        atom="Co 6.844708500129253, 5.772690500054138, 5.0;O 8.228239500226193, 6.352208500094744, 5.0;C 9.29272350030078, 6.822059500127666, 5.0; O 10.323201500372985, 7.276646500159519, 5.0;",
        basis="sto3g",
        charge=1,
        spin=0,
        unit=DistanceUnit.ANGSTROM,
    )
else:
    driver = PySCFDriver(
        atom="H 0 0 0; H 0 0 0.735",
        basis="sto3g",
        charge=0,
        spin=0,
        unit=DistanceUnit.ANGSTROM,
    )
problem = driver.run()
print(problem)

hamiltonian = problem.hamiltonian

coefficients = hamiltonian.electronic_integrals
print(coefficients.alpha)

second_q_op = hamiltonian.second_q_op()
print(second_q_op)

print(hamiltonian.nuclear_repulsion_energy)

Polynomial Tensor
 "+-":
[[-3.65678187e+02  4.38891446e+00  5.03645545e-03 ... -8.64216455e-05
  -1.11958108e-02 -2.17674388e-02]
 [ 4.38891446e+00 -9.46993900e+01  3.54010126e-02 ...  1.88580259e-04
   6.64761640e-02  8.15048831e-02]
 [ 5.03645545e-03  3.54010126e-02 -9.28127920e+01 ... -1.70447592e-02
  -2.26209123e-02 -2.30132117e-02]
 ...
 [-8.64216455e-05  1.88580259e-04 -1.70447592e-02 ... -3.34482975e+01
   6.54138544e-03  3.56471588e-03]
 [-1.11958108e-02  6.64761640e-02 -2.26209123e-02 ...  6.54138544e-03
  -2.66433646e+01  2.16509650e+00]
 [-2.17674388e-02  8.15048831e-02 -2.30132117e-02 ...  3.56471588e-03
   2.16509650e+00 -1.71741642e+01]]
 "++--":
[[[[ 1.63742811e+01 -2.35019312e+00  9.67521081e-04 ...  4.87631554e-05
     6.46422225e-03  1.26125097e-02]
   [-2.35019312e+00  5.78799165e-01 -1.17447565e-04 ... -1.22008224e-05
    -1.63518385e-03 -3.18211387e-03]
   [ 9.67521081e-04 -1.17447565e-04  2.74014758e-01 ...  1.29882093e-04
     3.47313602e-04  4.52674219e-04]
   

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
from qiskit_nature.second_q.algorithms import GroundStateEigensolver, NumPyMinimumEigensolverFactory
from qiskit_nature.second_q.mappers import JordanWignerMapper, QubitConverter

solver = GroundStateEigensolver(
    QubitConverter(JordanWignerMapper()),
    NumPyMinimumEigensolverFactory(),
)

result = solver.solve(problem)
print(result)

